In [31]:
import Pkg; Pkg.add("MLDatasets")
Pkg.add("OneHotArrays")
using Flux, Statistics, DataFrames, OneHotArrays
using StatsBase
using CSV
using Plots
using MLBase
Pkg.offline(true)



   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise5/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise5/Manifest.toml`
┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("1285c0f1-ff9f-5867-b66e-0f359bcf09ba"), "SpecialFunctionsExt")
│   Base.PkgId(Base.UUID("a1ab010e-6f71-51d6-acc8-fb8bfa4c0f92"), "FluxCUDAcuDNNExt")
│   Base.PkgId(Base.UUID("63c18a36-062a-441e-b654-da1e3ab1ce7c"), "KernelAbstractions")
│   Base.PkgId(Base.UUID("dd41ee52-2073-581e-92e8-26baf003f19a"), "FluxCUDAExt")
│   Base.PkgId(Base.UUID("6eaa3634-1823-5a95-8160-0f368f727af8"), "StridedViewsCUDAExt")
│   Base.PkgId(Base.UUID("0b1bfda6-eb8a-41d2-88d8-f5af5cad476f"), "OneHotArrays")
│   Base.PkgId(Base.UUID("338a49bd-c049-5b4c-976b-5ff9d2bfb7e7"), "MLDataDevicesOneHotArraysExt")
│   Base.PkgId(Base.UUID("02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"), "cuDNN")
│   Base.PkgId(Base.UUID("c8a29c40-e0ac-5e6e-8984-8da3115ac879"), "NNlibEnzym

confusion_matrix (generic function with 1 method)

In [62]:


function findaccuracy(predictions, actual)
    correct = sum(predictions .== actual)
    accuracy = correct / length(actual)
    return accuracy
end

function confusion_matrix(actual, predicted)
    # Obtener las etiquetas únicas
    labels = unique(vcat(actual, predicted))
    n_classes = length(labels)
    
    # Inicializar matriz de confusión
    conf_matrix = zeros(Int, n_classes, n_classes)
    
    # Llenar la matriz de confusión
    for (act, pred) in zip(actual, predicted)
        i = findfirst(==(act), labels)
        j = findfirst(==(pred), labels)
        conf_matrix[i,j] += 1
    end
    
    # Agregar encabezados
    println("\nMatriz de Confusión:")
    println("Predicho →")
    println("Real ↓")
    println("     ", join([string(i) for i in labels], "   "))
    for i in 1:n_classes
        print(labels[i], "  ")
        println(join([string(conf_matrix[i,j]) for j in 1:n_classes], "   "))
    end
    
    return conf_matrix
end

function hacer_roc_curve(gt, pred,numeroatributos)
    roc_curve = MLBase.roc(gt, pred)


    # Extraer los puntos de la curva ROC
    fpr = [r.fp / (r.fp + r.tn) for r in roc_curve]  # Tasa de Falsos Positivos
    tpr = [r.tp / (r.tp + r.fn) for r in roc_curve]  # Tasa de Verdaderos Positivos
    
    
    # Graficar la curva ROC
    plot(fpr, tpr,
        label="Curva ROC",
        xlabel="Tasa de Falsos Positivos (1-Especificidad)",
        ylabel="Tasa de Verdaderos Positivos (Sensibilidad)",
        title="Curva ROC",
        legend=:bottomright)
    
    
    # Agregar línea diagonal de referencia
    plot!([0, 1], [0, 1],
         label="Línea Base",
         linestyle=:dash,
         color=:gray)
    savefig("../fig/roc_curve_$(numeroatributos).png")
end


hacer_roc_curve (generic function with 2 methods)

In [114]:
C = CSV.read("../dat/Churn_Modelling.csv", DataFrame);
desc = describe(C)
numeroatributos = 5
# Verificar países únicos en Geography
países_únicos = unique(C.Geography)
println("Países únicos en el dataset: ", países_únicos)

# Crear columnas numéricas para Geography y Gender
C.Geography_num = map(x -> if x == "France" 
                            1 
                         elseif x == "Spain"
                            2
                         elseif x == "Germany"
                            3
                         end, C.Geography)

# Convertir Gender a valores numéricos (0 para Female, 1 para Male)
C.Gender_num = map(x -> x == "Male" ? 1 : 0, C.Gender)
if numeroatributos == 10
C=C[!, [:Geography_num, :Gender_num, :CreditScore, :Age, :Tenure, :Balance, :NumOfProducts, :HasCrCard, :IsActiveMember, :EstimatedSalary, :Exited]]  
elseif numeroatributos == 5
C=C[!, [ :Tenure, :Balance, :HasCrCard, :IsActiveMember, :EstimatedSalary, :Exited]]  
end

size(C)

Países únicos en el dataset: String7["France", "Spain", "Germany"]


(10000, 6)

In [115]:
#Elegir el conjunto de entrenamiento y prueba 
#80% para entrenamiento y 20% para prueba

@show size(C)
#obtener conjunto de datos para pruebas del código
# Definir el porcentaje deseado
porcentaje = 0.8 

# Calcular el número de índices a seleccionar
num_indices = Int(round(porcentaje * nrow(C)))

# Obtener índices aleatorios
indices_aleatorios = sample(1:nrow(C), num_indices, replace=false)

c_train = C[indices_aleatorios, :]
c_test = C[setdiff(1:nrow(C), indices_aleatorios), :]

#size(c_train,1) + size(c_test,1) 

size(C) = (10000, 6)


2000×6 DataFrame
  Row │ Tenure  Balance         HasCrCard  IsActiveMember  EstimatedSalary  Ex ⋯
      │ Int64   Float64         Int64      Int64           Float64          In ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │      9       0.0                1               1    14406.4           ⋯
    2 │      3       0.0                0               1     8487.75
    3 │      5       0.0                1               1        1.87616e5
    4 │      0   59697.2                1               1    53483.2
    5 │      3       0.0                1               0        1.40469e5     ⋯
    6 │      6       0.0                1               1    34410.6
    7 │      6   70349.5                0               1   178074.0
    8 │      8       1.22522e5          0               0        1.81298e5
  ⋮   │   ⋮           ⋮             ⋮            ⋮                ⋮            ⋱
 1994 │      1   90022.9                0               1     2988.28          ⋯
 1995 │      4       0.0                0               0        1.51325e5
 1996 │      4       0.0                1               0    49337.8
 1997 │      2       0.0                1               0        1.95192e5
 1998 │      7       1.5506e5           1               0    29179.5           ⋯
 1999 │     10   57369.6                1               1        1.017e5
 2000 │      7       0.0                0               1    42085.6
                                                  1 column and 1985 rows omitted

In [116]:

y = c_train[!, :Exited]
if numeroatributos == 10
    x = c_train[!, [:Geography_num, :Gender_num, :CreditScore, :Age, :Tenure, :Balance, :NumOfProducts, :HasCrCard, :IsActiveMember, :EstimatedSalary]]
elseif numeroatributos == 5
    x = c_train[!, [ :Tenure, :Balance, :HasCrCard, :IsActiveMember, :EstimatedSalary]]  
    end

@show x|> summary
@show y|> summary

x |> summary = "8000×5 DataFrame"
y |> summary = "8000-element Vector{Int64}"


"8000-element Vector{Int64}"

In [117]:
#Normaliza matriz de datos
x = Matrix(Float32.(x))
dt = fit(ZScoreTransform, x, dims=2)
x = StatsBase.transform(dt, x)

# Convertir valores numéricos a etiquetas de texto
y = map(val -> val == 0 ? "NO CANCELA" : "SI CANCELA", y)
# Convertir y a vector
y = vec(y)
# Transponer x para que tenga la forma correcta
x = x'

@show x|> summary
@show y|> summary


x |> summary = "5×8000 adjoint(::Matrix{Float32}) with eltype Float32"
y |> summary = "8000-element Vector{String}"


"8000-element Vector{String}"

In [118]:
y_test = c_test[!, :Exited]
if numeroatributos == 10
    x_test = c_test[!, [:Geography_num, :Gender_num, :CreditScore, :Age, :Tenure, :Balance, :NumOfProducts, :HasCrCard, :IsActiveMember, :EstimatedSalary]]

elseif numeroatributos == 5
    x_test = c_test[!, [:Tenure, :Balance, :HasCrCard, :IsActiveMember, :EstimatedSalary]] 
end

# Normaliza matriz de datos de prueba
x_test = Matrix(Float32.(x_test))
dt = fit(ZScoreTransform, x_test, dims=2)
x_test = StatsBase.transform(dt, x_test)


y_test = map(val -> val == 0 ? "NO CANCELA" : "SI CANCELA", y_test)
y_test = vec(y_test)
x_test = x_test'
@show x_test|> summary
@show y_test|> summary

x_test |> summary = "5×2000 adjoint(::Matrix{Float32}) with eltype Float32"
y_test |> summary = "2000-element Vector{String}"


"2000-element Vector{String}"

In [119]:
# Crear vector de clases
const classes = ["SI CANCELA", "NO CANCELA"]
flux_y_onehot = onehotbatch(y, classes)

2×8000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  1  1  1  1  1  1  ⋅  1  1  1  1     1  1  1  1  ⋅  1  1  1  1  1  1  1

In [120]:
m(W, b, x) = W*x .+ b

m (generic function with 1 method)

In [121]:
W = rand(Float32, 2, numeroatributos);
b = [0.0f0, 0.0f0];

In [122]:
W

2×5 Matrix{Float32}:
 0.103043  0.262828  0.835317  0.492584  0.59707
 0.465231  0.226688  0.253611  0.794106  0.924431

In [123]:
flux_model = Chain(Dense(numeroatributos => 2), softmax)

Chain(
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
) 

In [124]:
flux_model[1].weight, flux_model[1].bias

(Float32[0.29499227 -0.5482038 … -0.5300841 0.47361207; 0.3147867 0.09309292 … -0.88369703 -0.7837895], Float32[0.0, 0.0])

In [125]:
function flux_loss(flux_model, features, labels_onehot)
    ŷ = flux_model(features)
    Flux.logitcrossentropy(ŷ, labels_onehot)
end;

In [126]:
flux_loss(flux_model, x, flux_y_onehot)

0.83039314f0

In [127]:

flux_accuracy(x, y) = mean(Flux.onecold(flux_model(x), classes) .== y);

In [128]:
flux_accuracy(x, y)

0.379125

In [129]:
function train_flux_model!(f_loss, model, features, labels_onehot)
    dLdm, _, _ = gradient(f_loss, model, features, labels_onehot)
    @. model[1].weight = model[1].weight - 0.1 * dLdm[:layers][1][:weight]
    @. model[1].bias = model[1].bias - 0.1 * dLdm[:layers][1][:bias]
end;
for i = 1:500
    train_flux_model!(flux_loss, flux_model, x, flux_y_onehot);
    flux_accuracy(x, y) >= 0.98 && break
end

@show flux_accuracy(x, y);


flux_accuracy(x, y) = 0.79575


In [130]:
flux_loss(flux_model, x, flux_y_onehot)

0.5197991f0

In [131]:
predict_ = flux_model(x_test)

2×2000 Matrix{Float32}:
 0.0187891  0.0187874  0.0187837  …  0.0256171  0.00763814  0.0187846
 0.981211   0.981213   0.981216      0.974383   0.992362    0.981215

In [132]:
predict_class = predict_[1,:]
y_predict  = map(val -> val >= 0.5 ? "SI CANCELA" : "NO CANCELA", predict_class)


2000-element Vector{String}:
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 ⋮
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"
 "NO CANCELA"

In [133]:
findaccuracy(y_predict, y_test)

0.7985

In [134]:
confusion_matrix(y_test, y_predict)


Matriz de Confusión:
Predicho →
Real ↓
     NO CANCELA   SI CANCELA
NO CANCELA  1597   0
SI CANCELA  403   0


2×2 Matrix{Int64}:
 1597  0
  403  0

In [135]:

gt = map(val -> val == "NO CANCELA" ? 0 : 1, y_test)

# Crear un vector con el índice del valor máximo entre las dos filas para cada columna
predict_max = []
for i in 1:size(predict_,2)
      # Asegúrate de que los valores sean de tipo Float
      val1 = Float32(predict_[1, i])
      val2 = Float32(predict_[2, i])
      push!(predict_max, max(val1, val2))
      #push!(predict_max, predict_[2, i])
 
end


println("Número de clientes que NO CANCELAN: ", count(x -> x == "NO CANCELA", y_predict))
println("Número de clientes que SI CANCELAN: ", count(x -> x == "SI CANCELA", y_predict))


Número de clientes que NO CANCELAN: 2000
Número de clientes que SI CANCELAN: 0


In [136]:


# Supongamos que tienes las etiquetas verdaderas y las predicciones


pred = vec(Float32.(predict_max))
hacer_roc_curve(gt, pred,numeroatributos)
# Calcular la curva ROC


"/home/cecilia/Desktop/Ejercicios/Exercise5/fig/roc_curve_5.png"